In [39]:
# === 필요한 라이브러리 ===
import re
from typing import Annotated, Literal, List, Optional
from typing_extensions import TypedDict

# LangGraph 핵심
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage

# Document 폴백 (langchain.schema.Document가 없을 때 대비)
try:
    from langchain.schema import Document  # type: ignore
except Exception:
    from dataclasses import dataclass
    @dataclass
    class Document:
        page_content: str
        metadata: dict

# === 간단한 벡터스토어 대체 (키워드 기반 '의미론적' 검색) ===
class SimpleKeywordVectorStore:
    """토큰 겹침(자카드 유사도)으로 흉내내는 경량 검색기."""
    def __init__(self):
        self._docs: List[Document] = []
        self._tokens: List[set] = []

    @staticmethod
    def _tokenize(text: str) -> set:
        # 한글/숫자/통화기호 정도만 남기고 토큰화
        text = re.sub(r"[^가-힣a-zA-Z0-9₩,.\s]", " ", text)
        toks = set(t.strip() for t in re.split(r"\s+", text) if t.strip())
        return toks

    def add_documents(self, docs: List[Document]) -> None:
        for d in docs:
            self._docs.append(d)
            self._tokens.append(self._tokenize(d.page_content + " " + " ".join(f"{k}:{v}" for k, v in d.metadata.items())))

    def similarity_search(self, query: str, k: int = 4) -> List[Document]:
        if not self._docs:
            return []
        q = self._tokenize(query)
        scores = []
        for i, toks in enumerate(self._tokens):
            inter = len(q & toks)
            union = len(q | toks) if q or toks else 1
            score = inter / union
            scores.append((score, i))
        scores.sort(reverse=True, key=lambda x: x[0])
        # 0점만 있는 경우는 공집합이므로 빈 결과
        top = [self._docs[i] for s, i in scores[:k] if s > 0]
        return top

# === 메뉴 정보 추출 (과제 제시 정규식 사용) ===
def extract_menu_info(doc: Document) -> dict:
    """Vector DB 문서에서 구조화된 메뉴 정보 추출"""
    content = doc.page_content
    menu_name = doc.metadata.get('menu_name', 'Unknown')

    price_match = re.search(r'₩([\d,]+)', content)
    description_match = re.search(r'설명:\s*(.+?)(?:\n|$)', content, re.DOTALL)

    return {
        "name": menu_name,
        "price": price_match.group(0) if price_match else "가격 정보 없음",
        "description": description_match.group(1).strip() if description_match else "설명 없음"
    }

# === 문의 유형 분류 ===
def classify_user_query(user_message: str) -> Literal["가격 문의", "추천 요청", "메뉴 문의", "기타"]:
    msg = user_message.strip()
    if "가격" in msg or "얼마" in msg or "비싸" in msg or "가격표" in msg:
        return "가격 문의"
    if "추천" in msg or "뭐가 좋아" in msg or "인기" in msg or "베스트" in msg:
        return "추천 요청"
    if "메뉴" in msg or "있나요" in msg or "파나요" in msg or "설명" in msg:
        return "메뉴 문의"
    return "기타"

# === 상태 정의 (LangGraph 권장 방식: TypedDict + add_messages) ===
class CafeState(TypedDict, total=False):
    # 대화 메시지 리스트 자동관리
    messages: Annotated[List[AnyMessage], add_messages]
    # 분기 라우트
    route: Literal["가격 문의", "추천 요청", "메뉴 문의", "기타"]
    # 검색 쿼리 기록
    last_query: str
    # ⭐⭐추가: 검색 결과를 저장하여 노드 간 전달하는 필드 정의 ⭐⭐
    search_results: List[dict] 

# === 데이터 소스 (벡터스토어) 준비 ===
MENU_DB = SimpleKeywordVectorStore()
MENU_DB.add_documents([
    Document(page_content="₩5,000\n설명: 달콤한 카라멜 마끼야또", metadata={"menu_name": "카라멜 마끼야또"}),
    Document(page_content="₩3,500\n설명: 부드러운 아메리카노", metadata={"menu_name": "아메리카노"}),
    Document(page_content="₩5,500\n설명: 향긋한 바닐라라떼", metadata={"menu_name": "바닐라 라떼"}),
    Document(page_content="₩4,800\n설명: 진한 카페라떼", metadata={"menu_name": "카페라떼"}),
])

# === 노드 구현 ===
def node_classify(state: CafeState) -> CafeState:
    """마지막 사용자 메시지로 문의 유형을 결정하고 route에 기록."""
    # 마지막 메시지 가져오기
    last_user = None
    for m in reversed(state["messages"]):
        if isinstance(m, HumanMessage):
            last_user = m
            break
    user_text = (last_user.content if last_user else "").strip()
    route = classify_user_query(user_text) if user_text else "기타"
    state["route"] = route
    state["last_query"] = user_text
    return state

# === 노드 구현 (수정) ===
def node_search(state: CafeState) -> CafeState:
    """문의 유형별 검색 전략 수행 후, 중간 결과(구조화) 메시지로 저장."""
    route = state.get("route", "기타")
    user_message = state.get("last_query", "")

    all_menu_names = " ".join([d.metadata['menu_name'] for d in MENU_DB._docs]) # 모든 메뉴 이름

    if route == "가격 문의":
        # (이전에 수정한 로직)
        docs = MENU_DB.similarity_search(f"메뉴 가격 {all_menu_names}", k=5) 
        
    elif route == "추천 요청":
        # ⭐⭐수정 1: 사용자 메시지에 모든 메뉴 이름을 추가하여 검색 쿼리 강화⭐⭐
        enhanced_query = user_message + " " + all_menu_names
        docs = MENU_DB.similarity_search(enhanced_query, k=3)
        
        if not docs:
            # ⭐⭐수정 2: 폴백 쿼리에도 모든 메뉴 이름을 추가⭐⭐
            docs = MENU_DB.similarity_search(f"인기 메뉴 {all_menu_names}", k=3)
            
    elif route == "메뉴 문의":
        docs = MENU_DB.similarity_search(user_message, k=4)
    else:
        docs = MENU_DB.similarity_search(user_message, k=3)

    if not docs:
        state["messages"] += [AIMessage(content="관련 정보를 찾지 못했습니다.")]
        # 검색 실패 시 빈 리스트로 초기화
        state["search_results"] = [] 
        return state

    # 상위 1~3개만 구조화해서 임시 메시지로 붙임
    infos = [extract_menu_info(d) for d in docs[:3]]
    pretty = "\n".join([f"- {i['name']} | {i['price']} | {i['description']}" for i in infos])
    state["messages"] += [AIMessage(content=f"[검색결과]\n{pretty}")]
    
    # ⭐⭐수정: 정의된 필드에 저장
    state["search_results"] = infos  
    return state

def node_respond(state: CafeState) -> CafeState:
    route = state.get("route", "기타")
    infos: List[dict] = state.get("search_results", [])
    query = state.get("last_query", "")

    if not infos:
        state["messages"] += [AIMessage(content="죄송합니다. 해당 요청을 처리할 수 없습니다.")]
        return state

    # ⭐⭐ 핵심 수정 1: 질문에서 특정 메뉴 이름 찾기 ⭐⭐
    target_menu_name = None
    known_menus = [d['name'] for d in infos] # 검색된 메뉴 목록
    
    # 질문에 검색된 메뉴 중 하나가 포함되어 있는지 확인
    for menu_name in known_menus:
        # 질문에 메뉴 이름이 완벽히 포함되어 있다면 특정 메뉴 질문으로 간주
        if menu_name in query: 
            target_menu_name = menu_name
            break

    is_specific_query = (target_menu_name is not None)
    top = infos[0] # 기본값은 검색 결과 1위

    if is_specific_query:
        # 특정 메뉴가 언급되었다면, 해당 메뉴 정보를 top으로 설정 (응답의 정확성 확보)
        for info in infos:
            if info['name'] == target_menu_name:
                top = info
                break
    
    # ⭐⭐ 핵심 수정 2: '가격 문의' 시 특정 메뉴 언급 여부에 따라 응답 분기 ⭐⭐
    if route == "가격 문의":
        if is_specific_query:
            # ➡️ 특정 메뉴 가격 요청: 해당 메뉴만 응답
            msg = f"{top['name']}의 가격은 {top['price']}입니다."
        else:
            # ➡️ 일반 가격 목록 요청: 모든 검색 결과를 응답
            price_list = []
            for info in infos: 
                price_list.append(f"{info['name']}은 {info['price']}")
            msg = "문의하신 관련 메뉴들의 가격 정보입니다:\n" + "\n".join(price_list)
            
    elif route == "추천 요청":
        msg = f"추천 메뉴: {top['name']} — {top['description']} ({top['price']})."
        
    elif route == "메뉴 문의":
        msg = f"{top['name']} 설명: {top['description']} (가격: {top['price']})."

    else: # route == "기타"
        msg = f"요청에 맞는 정보를 찾지 못했습니다. 관련 메뉴: {top['name']} — {top['description']} ({top['price']})."

    state["messages"] += [AIMessage(content=msg)]
    return state

# === 그래프 구성 ===
graph = StateGraph(CafeState)
graph.add_node("classify", node_classify)
graph.add_node("search", node_search)
graph.add_node("respond", node_respond)

graph.add_edge(START, "classify")
graph.add_edge("classify", "search")
graph.add_edge("search", "respond")
graph.add_edge("respond", END)

app = graph.compile()

# === 개선된 검색 함수 ===
def semantic_search(menu_db, user_message):
    """키워드와 유사도를 기반으로 검색. 좀 더 정확하게 검색할 수 있도록 개선."""
    # 유사도를 테스트하기 위한 기본 키워드 검색. 수정 및 개선 필요.
    query_tokens = set(user_message.split())  # 간단한 공백 기반 토큰화 (확장 가능)
    
    # 예시로 각 문서에 대해 유사도를 계산하고 점수 반환
    scores = []
    for doc in menu_db._docs:
        doc_tokens = set(doc.page_content.split())  # 간단한 공백 기준 토큰화
        common_tokens = query_tokens & doc_tokens  # 교집합 (자카드 유사도)
        score = len(common_tokens) / (len(query_tokens) + len(doc_tokens) - len(common_tokens))  # 자카드 유사도 계산
        scores.append((score, doc))
    
    # 점수가 높은 상위 4개의 문서 반환
    scores.sort(reverse=True, key=lambda x: x[0])
    top_docs = [doc for score, doc in scores[:4]]  # 상위 4개 문서
    return top_docs

# === 사용 예시 ===
state: CafeState = {
    #  "messages": [HumanMessage(content="이 카페의 메뉴 가격을 알고 싶어요.")]
      "messages": [HumanMessage(content="이 카페에서 제일 인기 많은 메뉴를 추천해 줄래?")]
    #  "messages": [HumanMessage(content="바닐라 라떼 가격이 얼마예요?")]
}
result = app.invoke(state)
for m in result["messages"]:
    role = "USER" if isinstance(m, HumanMessage) else "ASSISTANT"
    print(f"[{role}] {m.content}")

[USER] 이 카페에서 제일 인기 많은 메뉴를 추천해 줄래?
[ASSISTANT] [검색결과]
- 카라멜 마끼야또 | ₩5,000 | 달콤한 카라멜 마끼야또
- 바닐라 라떼 | ₩5,500 | 향긋한 바닐라라떼
- 아메리카노 | ₩3,500 | 부드러운 아메리카노
[ASSISTANT] 추천 메뉴: 카라멜 마끼야또 — 달콤한 카라멜 마끼야또 (₩5,000).
